# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 25 2022 1:11PM,249354,10,PBR-102122-04,"Senseonics, Incorporated",Released
1,Oct 25 2022 1:08PM,249351,10,PBR-102122-02,"Senseonics, Incorporated",Released
2,Oct 25 2022 1:07PM,249358,19,ACG-SAMPLE-SERAN,ACG North America LLC,Released
3,Oct 25 2022 1:05PM,249350,10,PBR-102122-01,"Senseonics, Incorporated",Released
4,Oct 25 2022 1:04PM,249356,19,ADV80006935,"AdvaGen Pharma, Ltd",Released
5,Oct 25 2022 1:04PM,249356,19,ADV80006936,"AdvaGen Pharma, Ltd",Released
6,Oct 25 2022 1:04PM,249356,19,ADV80006937,"AdvaGen Pharma, Ltd",Released
7,Oct 25 2022 1:04PM,249356,19,ADV80006938,"AdvaGen Pharma, Ltd",Released
8,Oct 25 2022 1:04PM,249356,19,ADV80006939,"AdvaGen Pharma, Ltd",Released
9,Oct 25 2022 1:04PM,249356,19,ADV80006947,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,249353,Released,5
22,249354,Released,1
23,249355,Released,4
24,249356,Released,18
25,249358,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249353,NaN,NaN,5.0
249354,NaN,NaN,1.0
249355,NaN,NaN,4.0
249356,NaN,NaN,18.0
249358,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,0.0,1.0,0.0
249293,27.0,19.0,16.0
249299,0.0,14.0,29.0
249301,0.0,0.0,1.0
249302,0.0,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,0,1,0
249293,27,19,16
249299,0,14,29
249301,0,0,1
249302,0,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249279,0,1,0
1,249293,27,19,16
2,249299,0,14,29
3,249301,0,0,1
4,249302,0,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249279,,1,
1,249293,27,19,16
2,249299,,14,29
3,249301,,,1
4,249302,,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated"
1,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated"
2,Oct 25 2022 1:07PM,249358,19,ACG North America LLC
3,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated"
4,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd"
22,Oct 25 2022 1:02PM,249355,10,"Methapharm, Inc."
23,Oct 25 2022 1:02PM,249355,10,Methapharm-G
26,Oct 25 2022 1:01PM,249353,12,Hush Hush
31,Oct 25 2022 1:00PM,249352,20,"Exact-Rx, Inc."
38,Oct 25 2022 12:43PM,249348,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",,,1
1,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",,,1
2,Oct 25 2022 1:07PM,249358,19,ACG North America LLC,,,1
3,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",,,1
4,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",,,18
5,Oct 25 2022 1:02PM,249355,10,"Methapharm, Inc.",,,4
6,Oct 25 2022 1:02PM,249355,10,Methapharm-G,,,4
7,Oct 25 2022 1:01PM,249353,12,Hush Hush,,,5
8,Oct 25 2022 1:00PM,249352,20,"Exact-Rx, Inc.",,,7
9,Oct 25 2022 12:43PM,249348,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1,,
1,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1,,
2,Oct 25 2022 1:07PM,249358,19,ACG North America LLC,1,,
3,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",1,,
4,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",18,,
5,Oct 25 2022 1:02PM,249355,10,"Methapharm, Inc.",4,,
6,Oct 25 2022 1:02PM,249355,10,Methapharm-G,4,,
7,Oct 25 2022 1:01PM,249353,12,Hush Hush,5,,
8,Oct 25 2022 1:00PM,249352,20,"Exact-Rx, Inc.",7,,
9,Oct 25 2022 12:43PM,249348,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1,,
1,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1,,
2,Oct 25 2022 1:07PM,249358,19,ACG North America LLC,1,,
3,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",1,,
4,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",18,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Oct 25 2022 1:07PM,249358,19,ACG North America LLC,1.0,NaN,NaN
3,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",18.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 1:11PM,249354,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Oct 25 2022 1:08PM,249351,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Oct 25 2022 1:07PM,249358,19,ACG North America LLC,1.0,0.0,0.0
3,Oct 25 2022 1:05PM,249350,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Oct 25 2022 1:04PM,249356,19,"AdvaGen Pharma, Ltd",18.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1994773,14.0,30.0,6.0
12,498632,5.0,1.0,0.0
15,747894,52.0,33.0,27.0
19,1246753,22.0,0.0,0.0
20,498667,7.0,1.0,0.0
21,249301,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1994773,14.0,30.0,6.0
1,12,498632,5.0,1.0,0.0
2,15,747894,52.0,33.0,27.0
3,19,1246753,22.0,0.0,0.0
4,20,498667,7.0,1.0,0.0
5,21,249301,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,14.0,30.0,6.0
1,12,5.0,1.0,0.0
2,15,52.0,33.0,27.0
3,19,22.0,0.0,0.0
4,20,7.0,1.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,14.0
1,12,Released,5.0
2,15,Released,52.0
3,19,Released,22.0
4,20,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,6.0,0.0,27.0,0.0,0.0,0.0
Executing,30.0,1.0,33.0,0.0,1.0,0.0
Released,14.0,5.0,52.0,22.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,6.0,0.0,27.0,0.0,0.0,0.0
1,Executing,30.0,1.0,33.0,0.0,1.0,0.0
2,Released,14.0,5.0,52.0,22.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,6.0,0.0,27.0,0.0,0.0,0.0
1,Executing,30.0,1.0,33.0,0.0,1.0,0.0
2,Released,14.0,5.0,52.0,22.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()